# The Library

In [3]:
import pandas as pd
import os

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# The Necessary Files

In [4]:
current_dir = os.getcwd()
folder = '/data/results_international/'
result_file = 'adj_final_results.csv'
rubicology_file = 'all_games_RWC2023_fixed.csv'

# First exploration: Any upsets?

In [ ]:
#first get the rating of the teams
